In [5]:
library(tidyverse)
library(tidymodels)
library(repr)
library(ggplot2)
library(lubridate)
library(forcats)
library(tidyr)

In [6]:
players <- read_csv("data/players.csv")

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [7]:
players_cleaned <- players |> filter(!is.na(experience), !is.na(subscribe), !is.na(hashedEmail), !is.na(played_hours), !is.na(name), !is.na(gender), !is.na(Age))

In [38]:
sessions_player <- players_cleaned |> group_by(hashedEmail) |> summarize(number_of_sessions = n(), .groups = "drop")
players_combined <- players_cleaned |> left_join (sessions_player, by = "hashedEmail") |> filter (!is.na(number_of_sessions))
players_combined$experience <- as.factor(players_combined$experience)
players_final <- players_combined |> select(played_hours, number_of_sessions, subscribe, experience) |> mutate(subscribe = as.factor(subscribe))
players_final

played_hours,number_of_sessions,subscribe,experience
<dbl>,<int>,<fct>,<fct>
30.3,1,TRUE,Pro
3.8,1,TRUE,Veteran
0.0,1,FALSE,Veteran
0.7,1,TRUE,Amateur
0.1,1,TRUE,Regular
0.0,1,TRUE,Amateur
0.0,1,TRUE,Regular
0.0,1,FALSE,Amateur
0.1,1,TRUE,Amateur


In [34]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 5) |> set_engine("kknn") |> set_mode("classification")
knn_fit <- knn_spec |> fit(subscribe ~ experience + played_hours + number_of_sessions, data = players_final)
player_recipe <- recipe(subscribe ~ experience + played_hours + number_of_sessions, data = players_final) |> step_scale(played_hours)|>
step_center(played_hours) |> prep()
scaled_player <- bake(player_recipe, player_combined)
head(scaled_player)

ERROR: [1m[33mError[39m in `check_outcome()`:[22m
[33m![39m For a classification model, the outcome should be a `factor`, not a `logical`.


In [ ]:
scaled_player_graph <- scaled_player |> ggplot(aes